# Housing Prices Competition

By: Tony Zheng

## Imports/Packages

In [1]:
import pandas as pd
import sklearn as skl
import xgboost

## Reading/Pre-processing

In [2]:
data = pd.read_csv("train.csv", index_col="Id")
y = data["SalePrice"]
X = data.drop("SalePrice", axis=1)
n_train = X.shape[0]

### Filling NA

Let's see the amount of NA entries in each col

In [3]:
missing_cols = []
per_na = X.isna().sum(axis=0) / n_train * 100

for col in per_na.index:
    if per_na[col] != 0: 
        missing_cols.append(col)
        print(f"{col:<15} {X[col].dtype} {"":<15} {round(per_na[col], 1)}%")

LotFrontage     float64                 17.7%
Alley           object                 93.8%
MasVnrType      object                 59.7%
MasVnrArea      float64                 0.5%
BsmtQual        object                 2.5%
BsmtCond        object                 2.5%
BsmtExposure    object                 2.6%
BsmtFinType1    object                 2.5%
BsmtFinType2    object                 2.6%
Electrical      object                 0.1%
FireplaceQu     object                 47.3%
GarageType      object                 5.5%
GarageYrBlt     float64                 5.5%
GarageFinish    object                 5.5%
GarageQual      object                 5.5%
GarageCond      object                 5.5%
PoolQC          object                 99.5%
Fence           object                 80.8%
MiscFeature     object                 96.3%


Let's start with the float columns missing stuff.

LotFrontage and MasVnrArea both are NA presumably if the house does not have those things. In those cases, we'll just impute 0.

For GarageYrBlt, however, this is a bit tougher; we can't really encode in a float the garage's year if it doesn't exist. We'll just impute the knn for the column instead. Since there's a column already indicating if this value was missing, that's all we'll to do.


In [4]:
zero_cols = ["LotFrontage", "MasVnrArea"]
knn_cols = ["GarageYrBlt"]

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

fill_na = ColumnTransformer([
    ("Mean", KNNImputer(n_neighbors=7), knn_cols),
    ("Zero", SimpleImputer(strategy="constant", fill_value=0), zero_cols)
], remainder="passthrough", verbose_feature_names_out=False)

### Feature Engineering / Standardizing

#### Object Columns
We'll one hot where there are few enough categories, and ordinal encode when there are more.

In [6]:
ordinal_cols = []
oh_cols = []
max_unique = 15

for col in X.columns:
    values = X[col]

    if values.dtype == object:
        if len(values.unique()) > max_unique:
            ordinal_cols.append(col)
        
        else:
            oh_cols.append(col)

print(ordinal_cols)
print(oh_cols)

['Neighborhood', 'Exterior2nd']
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


#### Numerical Columns

Some of the numerical categories don't actually have any numerical meaning; we'll distinguish these categories by the number of unique values among them. Let's see the number of unique values for each column.

In [7]:
numerical_cols = []

for col in X.columns:
    values = X[col]
    if values.dtype != object:
        numerical_cols.append(col)

print(numerical_cols)

for column in numerical_cols:
    print(f"{column.ljust(10)} \t {len(X[column].unique())}")

['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']
MSSubClass 	 15
LotFrontage 	 111
LotArea    	 1073
OverallQual 	 10
OverallCond 	 9
YearBuilt  	 112
YearRemodAdd 	 61
MasVnrArea 	 328
BsmtFinSF1 	 637
BsmtFinSF2 	 144
BsmtUnfSF  	 780
TotalBsmtSF 	 721
1stFlrSF   	 753
2ndFlrSF   	 417
LowQualFinSF 	 24
GrLivArea  	 861
BsmtFullBath 	 4
BsmtHalfBath 	 3
FullBath   	 4
HalfBath   	 3
BedroomAbvGr 	 8
KitchenAbvGr 	 4
TotRmsAbvGrd 	 12
Fireplaces 	 4
GarageYrBlt 	 98
GarageCars 	 5
GarageArea 	 441
WoodDeckSF 	 274
OpenPorchSF 	 202
EnclosedP

By the looks of it, 10 seems to be a fairly reasonable threshold for categorical vs. numerical. Let's adjust for that...

In [8]:
oh_cols = oh_cols
standard_cols = []
thresh = 10

for col in numerical_cols:
    values = X[col]

    if len(values.unique()) > thresh: standard_cols.append(col)
    else: oh_cols.append(col)

print(standard_cols)
print(oh_cols)

['MSSubClass', 'LotFrontage', 'LotArea', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'TotRmsAbvGrd', 'GarageYrBlt', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'MiscVal', 'MoSold']
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition', 'OverallQual', 'OverallCond', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 

#### Creating the pipeline...

Since some columns overlap with each other when being transformed, we'll transform the column strings to be indices instead.

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

transform = ColumnTransformer([
    ("One Hot", OneHotEncoder(
            handle_unknown="infrequent_if_exist", sparse_output=False
        ), oh_cols),
    ("Standardize", StandardScaler(), standard_cols),
    ("Ordinal", OrdinalEncoder(
            handle_unknown="use_encoded_value", unknown_value=-1
        ), ordinal_cols)
], remainder="passthrough", verbose_feature_names_out=False)

fill_na.set_output(transform="pandas")
transform.set_output(transform="pandas")

process = Pipeline([
    ("Fill NA", fill_na),
    ("transform", transform)
])

process

Pipeline(steps=[('Fill NA',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Mean',
                                                  KNNImputer(n_neighbors=7),
                                                  ['GarageYrBlt']),
                                                 ('Zero',
                                                  SimpleImputer(fill_value=0,
                                                                strategy='constant'),
                                                  ['LotFrontage',
                                                   'MasVnrArea'])],
                                   verbose_feature_names_out=False)),
                ('transform',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('One Hot',
                                                  OneHotEncoder(han...
                                                   'BsmtUnfSF', 'TotalBsmtSF',
                                                   '1stFlrSF', '2ndFlrSF',
                                                   'LowQualFinSF', 'GrLivArea',
                                                   'TotRmsAbvGrd',
                                                   'GarageYrBlt', 'GarageArea',
                                                   'WoodDeckSF', 'OpenPorchSF',
                                                   'EnclosedPorch', '3SsnPorch',
                                                   'ScreenPorch', 'MiscVal',
                                                   'MoSold']),
                                                 ('Ordinal',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['Neighborhood',
                                                   'Exterior2nd'])],
                                   verbose_feature_names_out=False))])

## Making the Model

First, let's copy Kaggle's loss function for reference

In [10]:
from sklearn.metrics import make_scorer
import numpy as np

def lrmse(y, y_pred, **kwargs):
    return -((np.log(y) - np.log(y_pred)) ** 2 / len(y)).sum()
kaggle_loss= make_scorer(
    score_func=lrmse
)

And now we can do cross evaluation on some training data

In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size = .8)

from sklearn.base import clone
from sklearn.model_selection import cross_val_score

optimal = (-1, -1)
min_error = float('inf')

for max_depth in range(2, 14):
    for eta in [i / 100 for i in range(1, 10)]:
        new_process = clone(process)
        new_process.steps.append(("Model", xgboost.XGBRegressor(max_depth=max_depth, eta=eta, device = "cuda")))

        errors = cross_val_score(new_process, X_train, y_train, cv=5, scoring=kaggle_loss)
        mean = -sum(errors) / len(errors)

        if mean < min_error:
            min_error = mean
            optimal = (max_depth, eta)
        
        print(f"RMSE with d={max_depth:<3} & eta={eta:>3}: {mean}")

RMSE with d=2   & eta=0.01: 0.07531814625312223
RMSE with d=2   & eta=0.02: 0.04828360697951875
RMSE with d=2   & eta=0.03: 0.03757113043355944
RMSE with d=2   & eta=0.04: 0.032067053090055544
RMSE with d=2   & eta=0.05: 0.02901391306108239
RMSE with d=2   & eta=0.06: 0.026721314065924096
RMSE with d=2   & eta=0.07: 0.025502133397603786
RMSE with d=2   & eta=0.08: 0.02474578784422519
RMSE with d=2   & eta=0.09: 0.023307488182449387
RMSE with d=3   & eta=0.01: 0.06517490662702848
RMSE with d=3   & eta=0.02: 0.03882120816033377
RMSE with d=3   & eta=0.03: 0.02993404582030946
RMSE with d=3   & eta=0.04: 0.02586703677337603
RMSE with d=3   & eta=0.05: 0.023877562660155054
RMSE with d=3   & eta=0.06: 0.02249853294474602
RMSE with d=3   & eta=0.07: 0.021548292742975945
RMSE with d=3   & eta=0.08: 0.021106097464022897
RMSE with d=3   & eta=0.09: 0.020212675700597572
RMSE with d=4   & eta=0.01: 0.06020322325262386
RMSE with d=4   & eta=0.02: 0.03451246704104142
RMSE with d=4   & eta=0.03: 0.02

Let's see what the optimal hyperparameters are...

In [12]:
optimal

(5, 0.09)

And now see how they do on the validation data...

In [23]:
final_process = clone(process)
final_process.steps.append(("Model", xgboost.XGBRegressor(
    max_depth=optimal[0], 
    eta=optimal[1], 
    device = "cuda")
))

final_process.fit(X, y)
y_pred = final_process.predict(X_valid)
-lrmse(y=y_valid, y_pred=y_pred)

0.0031645493989411306

And finally output this to a csv

In [29]:
test_X = pd.read_csv("test.csv", index_col="Id")
index_col = test_X.index
predictions = pd.DataFrame(final_process.predict(test_X), index=index_col, columns=["SalePrice"])
predictions

,SalePrice
Id,
1461,121526.921875
1462,156252.875000
1463,188132.921875
1464,198888.921875
1465,187820.343750
...,...
2915,81463.054688
2916,88367.078125
2917,159652.203125


In [30]:
predictions.to_csv("submission.csv")